In [ ]:
from datetime import datetime
start = datetime.now()

import os
bucket = os.getenv("WORKSPACE_BUCKET")
bucket

import hail as hl
hl.init(default_reference='GRCh38', idempotent=True)

mt_wgs_path = os.getenv("WGS_HAIL_STORAGE_PATH")
mt_wgs_path

In [ ]:
mt = hl.read_matrix_table(mt_wgs_path)

In [ ]:
import os
import pandas as pd
from datetime import datetime

In [ ]:
start = datetime.now()

In [ ]:
bucket = os.getenv("WORKSPACE_BUCKET")
bucket

In [ ]:
dataset = os.getenv("WORKSPACE_CDR")
dataset

In [ ]:
# This snippet assumes you run setup first

# This code copies file in your Google Bucket and loads it into a dataframe

# Replace 'test.csv' with THE NAME of the file you're going to download from the bucket (don't delete the quotation marks)
name_of_file_in_bucket = 'n3c_aou_cohort_ft.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{name_of_file_in_bucket}' .")

print(f'[INFO] {name_of_file_in_bucket} is successfully downloaded into your working space')
# save dataframe in a csv file in the same workspace as the notebook
cohort = pd.read_csv(name_of_file_in_bucket)
cohort.head()


In [ ]:
#cohort = pd.read_csv('n3c_aou_cohort.csv')

In [ ]:
people = tuple(cohort['person_id'])

In [ ]:
# WGS
person_sql = f"""
SELECT  person.person_id,
        p_gender_concept.concept_name as gender,
        p_ethnicity_concept.concept_name as ethnicity,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `{dataset}.person` person 
    LEFT JOIN
        `{dataset}.concept` p_gender_concept 
            on person.gender_concept_id = p_gender_concept.CONCEPT_ID 
    LEFT JOIN
        `{dataset}.concept` p_ethnicity_concept 
            on person.ethnicity_concept_id = p_ethnicity_concept.CONCEPT_ID 
    LEFT JOIN
        `{dataset}.concept` p_sex_at_birth_concept 
            on person.sex_at_birth_concept_id = p_sex_at_birth_concept.CONCEPT_ID  
    WHERE
        person.PERSON_ID IN (
            select
                person_id  
            from
                `{dataset}.cb_search_person` cb_search_person  
            where
                cb_search_person.person_id in (
                    select
                        person_id 
                    from
                        `{dataset}.cb_search_person` p 
                    where
                        has_whole_genome_variant = 1 
                ) 
            )
            and person.PERSON_ID in {people}"""

wgs_demog = pd.read_gbq(person_sql, dialect="standard")

wgs_demog.head(5)

In [ ]:
demographics = pd.get_dummies(wgs_demog.set_index(['person_id'])).reset_index()
demographics.head()

In [ ]:
# save phenotypes locally
phenotypes = (demographics[["person_id", "ethnicity_Hispanic or Latino", "sex_at_birth_Male"]]
              .rename(columns={'ethnicity_Hispanic or Latino': 'is_hispanic',
                              'sex_at_birth_Male': 'is_male'})
             )
for col in ['is_hispanic', 'is_male']:
    phenotypes[col] = phenotypes[col].astype(int)

phenotypes["person_id"] = phenotypes["person_id"].astype(str)
    
phenotypes.to_csv('long_covid_phenotypes.tsv', index=False, sep='\t')

# save phenotypes to the bucket
!gsutil cp 'long_covid_phenotypes.tsv' {bucket}/data/

In [ ]:
phenotypes.head()

In [ ]:
mt.count()

In [ ]:
test_intervals = ['chr3:45859597-45859598', 'chr6:31153649-31153650', 'chr19:4719431-4719432', 'chr21:33252612-33252613', 'chr3:101705614-101705615', 'chr6:41534945-41534946', 
'chr1:155203736-155203737', 'chr3:45838989-45838990', 'chr6:31153455-31153456', 'chr6:41522644-41522645', 'chr9:133273813-133273814', 'chr10:79946568-79946569', 'chr11:1219991-1219992',
'chr11:34507219-34507220', 'chr12:112936943-112936944', 'chr12:132564254-132564255', 'chr16:89196249-89196250', 'chr17:45707983-45707984', 'chr17:49863303-49863304', 'chr19:4719431-4719432', 
'chr19:10355447-10355448', 'chr19:50379362-50379363', 'chr21:33242905-33242906', 'chr1:155127096-155127097', 'chr3:45793925-45793926', 'chr3:101780431-101780432',                            
'chr6:33076153-33076154', 'chr9:133273813-133273814', 'chr12:112914354-112914355', 'chr19:4719431-4719432', 'chr19:48867352-48867353',
'chrX:15602217-15602218', 'chr6:41515652-41515653']
#'chr23:15602217-15602218'
#Added in new geen chr6:41515652-41515652

In [ ]:
mt = hl.filter_intervals(mt, [hl.parse_locus_interval(x,) for x in test_intervals])

In [ ]:
mt.count()

In [ ]:
phenotype_filename = "gs://fc-secure-467ef02a-b21a-4a58-8aed-51cf6ceafdca/data/long_covid_phenotypes.tsv"

In [ ]:
phenotypes = (hl.import_table(phenotype_filename,
                              types={'person_id':hl.tstr},
                              impute=True,
                              key='person_id'))

In [ ]:
mt = hl.split_multi_hts(mt)
mt.count()

In [ ]:
phenotypes

In [ ]:
phenotype_df = phenotypes.to_pandas()

In [ ]:
phenotype_df.head()

In [ ]:
mt = mt.semi_join_cols(phenotypes)
mt.count()

In [ ]:
out_path = f'{bucket}/data/test_plink'

In [ ]:
hl.export_plink(mt, out_path, ind_id = mt.s)

In [ ]:
!gsutil ls {bucket}/data

In [ ]:
mt_rows =  mt.rows()

In [ ]:
mt_rows_df = mt.rows().to_pandas()

In [ ]:
!gsutil ls {bucket}/data

In [ ]:
array_plink_path = f'long-covid/plink'
array_plink_path

In [ ]:
genomic_location = os.getenv("CDR_STORAGE_PATH")
genomic_location

In [ ]:
!gsutil -u $GOOGLE_PROJECT ls gs://fc-secure-467ef02a-b21a-4a58-8aed-51cf6ceafdca/data/

In [ ]:
!gsutil -u $GOOGLE_PROJECT ls gs://fc-secure-467ef02a-b21a-4a58-8aed-51cf6ceafdca/data/

In [ ]:
!plink --bfile gs://fc-secure-467ef02a-b21a-4a58-8aed-51cf6ceafdca/data/test_plink --freq --out test_plink

In [ ]:
!plink --bfile {bucket}/data/test_plink --freq --out test_plink

In [ ]:
!plink --bfile gs://fc-secure-467ef02a-b21a-4a58-8aed-51cf6ceafdca/data/test_plink --freq --out test_plink

In [ ]:
from datetime import datetime
import os 
import pandas as pd

In [ ]:
start = datetime.now()
bucket = os.getenv("WORKSPACE_BUCKET")
bucket

In [ ]:
genomic_location = os.getenv("CDR_STORAGE_PATH")
genomic_location

In [ ]:
array_plink_path = 'gs://fc-secure-467ef02a-b21a-4a58-8aed-51cf6ceafdca/data/*'
array_plink_path

In [ ]:
!mkdir -p plink

In [ ]:
!gsutil ls {bucket}/data

In [ ]:
!gsutil -u $GOOGLE_PROJECT ls gs://fc-secure-467ef02a-b21a-4a58-8aed-51cf6ceafdca/data/test_plink.*

In [ ]:
!gsutil -u $$GOOGLE_PROJECT cp -r $array_plink_path plink/

In [ ]:
!ls plink/

In [ ]:
!plink -bfile plink/test_plink --freq --out plink/demo

In [ ]:
!head -n 10 plink/demo.frq

In [ ]:
!plink --bfile plink/test_plink --recode A --out plink/test_plink_recode

In [ ]:
!head -n 10 plink/test_plink_recode.raw

In [ ]:
import csv

#export your file.raw to tsv
with open('plink/test_plink_recode.raw') as infile, open('test_plink_recode.tsv', 'w') as outfile:
    lines = infile.readlines()
    for line in lines:
        outfile.write(line)

In [ ]:
gen_df = pd.read_table('test_plink_recode.tsv')

In [ ]:
gen_df

In [ ]:
for col in gen_df.columns:
    print(col)

In [ ]:
import subprocess

destination_filename = 'test_plink_recode.tsv'

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr


In [ ]:
#Change the output given above into a single dataframe with split columns
#This part was added by Chris 8/14/2023
old_column = gen_df.columns[0]
new_columns =  old_column.split()

seperated_df = pd.DataFrame(columns = new_columns)
seperated_df[new_columns]  = gen_df[gen_df.columns[0]].str.split(' ', expand= True)
#seperated_df[seperated_df['chr6:41515652:G:C_C']== "2"]#.unique()
#NA = 21, 0 = 4266, 1 = 639, 2 = 56

In [ ]:
seperated_df['person_id'] = seperated_df['IID']
seperated_df = seperated_df.drop(['FID','IID','PAT','MAT','SEX','PHENOTYPE'], axis=1)
seperated_df

In [ ]:
# This snippet assumes you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe = seperated_df  

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename = 'GeneticDataDfAugust14.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# save dataframe in a csv file in the same workspace as the notebook
my_dataframe.to_csv(destination_filename, index=False)

# get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file to the bucket
args = ["gsutil", "cp", f"./{destination_filename}", f"{my_bucket}/data/"]
output = subprocess.run(args, capture_output=True)

# print output from gsutil
output.stderr


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
x = np.array([4266, 695, 21])
xlables = ["No Expression", 'Expression', 'No data']
plt.pie(x, labels = xlables)
plt.show()

In [ ]:
x = np.array([0.5, 0.4, 0.1])
xlables = ["EHR Model", 'Survey Data Model', 'Genetic Model']
plt.title = "Contribution to Overall Prediction"
plt.pie(x, labels = xlables)
plt.show()